In [1]:
import numpy as np
from gurobipy import *
import import_ipynb
import pandas as pd
import statsmodels.api as sm

In [2]:
## Main Function Definition 
def optimize(T):

## generating parameters
    c = np.random.uniform(2000,5000,T)
    d = np.random.uniform(100,500,T)
    p = np.random.uniform(5,15,T)
    
## calculating b_t and h_t based on the generated p_t
    h =[]
    for j,n in np.ndenumerate(p):
        h.append((16-n)/5)
    ht=np.asarray(h, dtype=np) 
    b =[]
    for j,n in np.ndenumerate(p):
        b.append(20-n)
    bt =np.asarray(b,dtype=np)

## The model we use from Gurobi is Mixed Integer Programming
    m = Model("mip")
    x = range(T)

## adding decision variables for the model, production (x in the formula), 
## setup as binary variable (y in the formula, binary), inventory (i in the formula), 
## and lost demand (l in the formula)
    production = m.addVars(x,vtype=GRB.CONTINUOUS, name='production')
    setup = m.addVars(x, vtype=GRB.BINARY, name='setup')
    invent = m.addVars(x, vtype=GRB.CONTINUOUS, name='invent')
    lost = m.addVars(x, vtype=GRB.CONTINUOUS, name='lost')

## three types of constraints: demand, productivity, and lost demand upper bound   
    m.addConstrs((invent[t]+production[t+1] == invent[t+1]+d[t+1]+lost[t+1] 
              for t in x[1:T-1]),'demand')
## i_{t-1}+x_t = i_t + d_t + l_t
    m.addConstr((production[0]>=invent[0]+d[0]+lost[0]), 'demand(0)')
## i_{0}+x_1 = i_1 + d_1 + l_1
    m.addConstrs((production[t]<=500*T*setup[t] for t in x), 'production')
## x_t <= M*y_t
    m.addConstrs((lost[t]<=d[t] for t in x), 'lost')
## l_t <= d_t

## setting objective function
    m.setObjective(quicksum(p[t]*production[t] +
                            c[t]*setup[t] +
                            ht[t]*invent[t] +
                            bt[t]*lost[t] for t in x), GRB.MINIMIZE)

## Run the MIP optimization and report the result   
    m.optimize()
    for v in m.getVars():
        if v.X != 0:
            print("%s %f" % (v.Varname, v.X))

In [3]:
## Trial for T=100
optimize(100)

Using license file /Users/kenneth/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 299 rows, 400 columns and 695 nonzeros
Model fingerprint: 0x6be7ce18
Variable types: 300 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+04]
  Objective range  [2e-01, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 5e+02]
Found heuristic solution: objective 632993.77290
Presolve removed 120 rows and 50 columns
Presolve time: 0.01s
Presolved: 179 rows, 350 columns, 529 nonzeros
Found heuristic solution: objective 599909.76734
Variable types: 260 continuous, 90 integer (90 binary)

Root relaxation: objective 2.601358e+05, 166 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 260135.789    0   43 599909.767 260135.789  56.

In [4]:
## 1. For the multivariate regression

## Import the results for 20 experiments with T=100
df = pd.read_excel ('t100.xlsx') 

In [5]:
## 1. For the multivariate regression

## Define independent variables
X = df[['var_c', 'var_d','var_p']]

## Define dependent variable
y = df['unit cost']

## Adding Constant
X = sm.add_constant(X)

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

/Users/kenneth/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              unit cost   R-squared:                       0.389
Model:                            OLS   Adj. R-squared:                  0.274
Method:                 Least Squares   F-statistic:                     3.396
Date:                Fri, 11 Sep 2020   Prob (F-statistic):             0.0437
Time:                        14:23:43   Log-Likelihood:                -2.0142
No. Observations:                  20   AIC:                             12.03
Df Residuals:                      16   BIC:                             16.01
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         16.5328      1.591     10.390      0.000      13.160      19.906
var_c      -2.337e-06   1.31e-06     -1.786      0.093   -5.11e-06    4.37e-07
var_d      -8.275e-05   5.69e-05     -1.454      0.165      -0.000    3.79e-05
var_p         -0.2455      0.105     -2.344      0.032      -0.468      -0.023
==============================================================================
Omnibus:                        6.799   Durbin-Watson:                   2.500
Prob(Omnibus):                  0.033   Jarque-Bera (JB):                1.945
Skew:                           0.267   Prob(JB):                        0.378
Kurtosis:                       1.569   Cond. No.                     1.74e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.74e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:
## 2. For the 20*40 repeated experiments in order for average run time

## Define the main function
def optimize2(T):
    data = []
    list1 =[]
    for z in range(50, T, 50):
        list1.append(z)
## Here list1 will as follows, (50, 100, ..., T)

    res =  [ele for ele in list1 for p in range(20)] 
## Here res will be (50, 50, ..., 50, 100, 100,..., 100, ...., T, T) 

## generating parameters 
    for i in res:
        c = np.random.uniform(2000,5000,i)
        d = np.random.uniform(100,500,i)
        p = np.random.uniform(5,15,i)

## calculating b_t and h_t based on the generated p_t 
        h =[]
        for j,n in np.ndenumerate(p):
            h.append((16-n)/5)
        ht=np.asarray(h, dtype=np) 
        b =[]
        for j,n in np.ndenumerate(p):
            b.append(20-n)
        bt =np.asarray(b,dtype=np)

        m = Model("mip")
        x = range(i)

## adding decision variables for the model, production, setup as binary variable, inventory, and lost
        production = m.addVars(x,vtype=GRB.CONTINUOUS, name='production')
        setup = m.addVars(x, vtype=GRB.BINARY, name='setup')
        invent = m.addVars(x, vtype=GRB.CONTINUOUS, name='invent')
        lost = m.addVars(x, vtype=GRB.CONTINUOUS, name='lost')

## three types of constraints, demand, productivity, and lost demand upper bound
        m.addConstrs((invent[t]+production[t+1] == invent[t+1]+d[t+1]+lost[t+1] 
                  for t in x[1:i-1]),'demand')
        m.addConstr((production[0]>=invent[0]+d[0]+lost[0]), 'demand(1)')
        m.addConstrs((production[t]<=500*i*setup[t] for t in x), 'production')
        m.addConstrs((lost[t]<=d[t] for t in x), 'lost')

## setting objective function
        m.setObjective(quicksum(p[t]*production[t] +
                                c[t]*setup[t] +
                                ht[t]*invent[t] +
                                bt[t]*lost[t] for t in x), GRB.MINIMIZE)

        m.optimize()
## Sum of Production
        sumofpro = sum(m.x[:i])
## run time for each experiment
        runtime = m.Runtime

## Append the four variables to the dataframe, T, Total Cost, Sum of Production, and run time
        data.append((i, m.objVal, sumofpro, runtime))

    colums = ['times', 'total cost', 'total production', 'runtime']

    df = pd.DataFrame(data, columns=colums)

    print(df)
    
    df.to_excel("output2.xlsx") 

In [ ]:
## Trial run
optimize2(2050)

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 149 rows, 200 columns and 345 nonzeros
Model fingerprint: 0x4097916b
Variable types: 150 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [2e-01, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 5e+02]
Found heuristic solution: objective 298553.93676
Presolve removed 68 rows and 47 columns
Presolve time: 0.00s
Presolved: 81 rows, 153 columns, 234 nonzeros
Found heuristic solution: objective 264493.87607
Variable types: 112 continuous, 41 integer (41 binary)

Root relaxation: objective 1.144172e+05, 80 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 114417.249    0   21 264493.876 114417.249  56.7%     -    0s
H    0     0                    182590.54552 114417.249  37.3%     -    0s
     0 

H    0     0                    190507.98538 135434.534  28.9%     -    0s
     0     0 156987.251    0   15 190507.985 156987.251  17.6%     -    0s
     0     0 158577.656    0   13 190507.985 158577.656  16.8%     -    0s
     0     0 163257.473    0   11 190507.985 163257.473  14.3%     -    0s
H    0     0                    174349.34912 163257.473  6.36%     -    0s
     0     0 163412.887    0   11 174349.349 163412.887  6.27%     -    0s
     0     0 164537.646    0    3 174349.349 164537.646  5.63%     -    0s
H    0     0                    171581.74469 164537.646  4.11%     -    0s
H    0     0                    170263.17262 164537.646  3.36%     -    0s
     0     0 165028.173    0    7 170263.173 165028.173  3.07%     -    0s
     0     0 165529.800    0    3 170263.173 165529.800  2.78%     -    0s
H    0     0                    165714.66512 165529.800  0.11%     -    0s

Cutting planes:
  Gomory: 6
  Implied bound: 14
  MIR: 8
  Flow cover: 37
  Flow path: 6

Explored 

     0     0 156339.709    0   16 179959.908 156339.709  13.1%     -    0s
     0     0 156351.617    0   16 179959.908 156351.617  13.1%     -    0s
     0     0 157193.739    0   19 179959.908 157193.739  12.7%     -    0s
H    0     0                    164604.43995 157193.739  4.50%     -    0s
     0     0 157205.139    0   15 164604.440 157205.139  4.50%     -    0s
     0     0 157830.130    0    9 164604.440 157830.130  4.12%     -    0s
H    0     0                    160993.93971 157830.130  1.97%     -    0s
     0     0 158122.739    0   10 160993.940 158122.739  1.78%     -    0s
     0     0 158173.222    0    8 160993.940 158173.222  1.75%     -    0s
     0     0 158196.411    0    8 160993.940 158196.411  1.74%     -    0s
     0     0 158343.598    0   10 160993.940 158343.598  1.65%     -    0s
     0     0 158389.399    0    9 160993.940 158389.399  1.62%     -    0s
     0     0 158389.399    0    9 160993.940 158389.399  1.62%     -    0s
     0     0 158389.399  

     0     0 160706.558    0    6 167407.940 160706.558  4.00%     -    0s
     0     0 160839.910    0    3 167407.940 160839.910  3.92%     -    0s
     0     0 161145.099    0    1 167407.940 161145.099  3.74%     -    0s
*    0     0               0    161329.92120 161329.921  0.00%     -    0s

Cutting planes:
  Gomory: 6
  Implied bound: 21
  MIR: 4
  Flow cover: 25
  Flow path: 12

Explored 1 nodes (206 simplex iterations) in 0.09 seconds
Thread count was 4 (of 4 available processors)

Solution count 8: 161330 167408 173542 ... 314637

Optimal solution found (tolerance 1.00e-04)
Best objective 1.613299212029e+05, best bound 1.613299212029e+05, gap 0.0000%
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 149 rows, 200 columns and 345 nonzeros
Model fingerprint: 0xfed84704
Variable types: 150 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [2e-01, 5e+03]
  Bounds range     [1e+00, 1e+00]
  

Variable types: 150 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [3e-01, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 5e+02]
Found heuristic solution: objective 303678.15998
Presolve removed 66 rows and 43 columns
Presolve time: 0.00s
Presolved: 83 rows, 157 columns, 240 nonzeros
Found heuristic solution: objective 274698.41502
Variable types: 115 continuous, 42 integer (42 binary)

Root relaxation: objective 1.218843e+05, 80 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 121884.316    0   19 274698.415 121884.316  55.6%     -    0s
H    0     0                    183096.90244 121884.316  33.4%     -    0s
     0     0 144937.204    0   25 183096.902 144937.204  20.8%     -    0s
     0     0 147536.856    0   19 183096.902 147536.856  19.4%     -    0s
   

...

*   70    40              20    2063123.1542 2061512.02  0.08%   2.5    0s
H   99    29                    2062869.4428 2061796.26  0.05%   2.7    0s
*  103    29              16    2062397.9971 2061796.26  0.03%   2.7    0s

Cutting planes:
  Gomory: 76
  Implied bound: 246
  MIR: 88
  Flow cover: 429
  Flow path: 153

Explored 151 nodes (3268 simplex iterations) in 0.56 seconds
Thread count was 4 (of 4 available processors)

Solution count 10: 2.0624e+06 2.06287e+06 2.06312e+06 ... 2.45391e+06

Optimal solution found (tolerance 1.00e-04)
Best objective 2.062397997087e+06, best bound 2.062397997087e+06, gap 0.0000%
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 1799 rows, 2400 columns and 4195 nonzeros
Model fingerprint: 0x986ff9d9
Variable types: 1800 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+05]
  Objective range  [2e-01, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 5e+02]
Found heu

H    0     0                    2046976.3431 2022046.33  1.22%     -    0s
H    0     0                    2043458.6464 2022046.33  1.05%     -    0s
     0     2 2022046.33    0   72 2043458.65 2022046.33  1.05%     -    0s
H   65    80                    2038693.8071 2025892.55  0.63%   5.1    0s
H   72    80                    2036917.9481 2025892.55  0.54%   4.9    0s
H   87    86                    2036671.5050 2025892.55  0.53%   4.4    0s
H   88    86                    2034384.0229 2025892.55  0.42%   4.4    0s
*   89    86              38    2034376.9796 2025892.55  0.42%   4.4    0s
H 1158   480                    2034091.2901 2029835.00  0.21%   5.7    1s
H 1243   510                    2034071.1955 2030041.19  0.20%   5.6    1s
H 1486   543                    2034071.1951 2030501.56  0.18%   5.6    1s

Cutting planes:
  Gomory: 67
  Implied bound: 72
  MIR: 141
  Flow cover: 513
  Flow path: 127

Explored 1506 nodes (13516 simplex iterations) in 1.87 seconds
Thread count wa

     0     0 2108212.00    0  133 2320265.48 2108212.00  9.14%     -    0s
     0     0 2108752.48    0  134 2320265.48 2108752.48  9.12%     -    0s
     0     0 2113909.76    0  111 2320265.48 2113909.76  8.89%     -    0s
H    0     0                    2203505.0199 2113909.76  4.07%     -    0s
     0     0 2118219.66    0  112 2203505.02 2118219.66  3.87%     -    0s
     0     0 2119247.30    0  111 2203505.02 2119247.30  3.82%     -    0s
     0     0 2123380.72    0   94 2203505.02 2123380.72  3.64%     -    0s
H    0     0                    2202516.2938 2123380.72  3.59%     -    0s
     0     0 2124831.29    0   94 2202516.29 2124831.29  3.53%     -    0s
     0     0 2124920.94    0   89 2202516.29 2124920.94  3.52%     -    0s
     0     0 2129001.57    0   57 2202516.29 2129001.57  3.34%     -    0s
     0     0 2129433.94    0   54 2202516.29 2129433.94  3.32%     -    0s
     0     0 2131677.17    0   45 2202516.29 2131677.17  3.22%     -    0s
H    0     0             

H    0     0                    2332730.6473 2202361.71  5.59%     -    0s
     0     0 2204740.19    0   96 2332730.65 2204740.19  5.49%     -    0s
     0     0 2204993.11    0   99 2332730.65 2204993.11  5.48%     -    0s
     0     0 2210078.77    0   69 2332730.65 2210078.77  5.26%     -    0s
H    0     0                    2237865.3914 2210078.77  1.24%     -    0s
     0     0 2210733.65    0   65 2237865.39 2210733.65  1.21%     -    0s
     0     0 2213108.49    0   38 2237865.39 2213108.49  1.11%     -    0s
     0     0 2213576.73    0   26 2237865.39 2213576.73  1.09%     -    0s
     0     0 2213994.69    0   26 2237865.39 2213994.69  1.07%     -    0s
H    0     0                    2224861.5994 2213994.69  0.49%     -    0s
     0     0 2214122.98    0   26 2224861.60 2214122.98  0.48%     -    0s
     0     0 2214547.82    0   16 2224861.60 2214547.82  0.46%     -    0s
H    0     0                    2222041.1712 2214547.82  0.34%     -    0s
     0     0 2214920.98  


Optimal solution found (tolerance 1.00e-04)
Best objective 2.180624418214e+06, best bound 2.180624418214e+06, gap 0.0000%
